# Transfer Learning — Atelier pratique

**Deep Learning par la pratique — Alexis Perrier**

---

## Objectifs de cet atelier

1. **Comprendre le Transfer Learning** : pourquoi on ne part jamais de zéro en deep learning
2. **Travailler avec l'IA** : utiliser Gemini pour générer, lire et comprendre du code

## Comment utiliser ce notebook

- Les cellules avec du code pré-écrit : **lisez, exécutez, observez**
- Les cellules marquées 🤖 **GEMINI** : demandez à Gemini de générer le code, puis lisez-le et demandez-lui de vous l'expliquer
- Les cellules marquées ❓ **QUESTION** : répondez en observant les résultats

**Utiliser l'IA pour coder n'est pas tricher — c'est la méthode de travail.**
Votre valeur n'est pas de taper du code, c'est de savoir quoi demander, comprendre ce qu'on vous donne, et juger si le résultat est bon.

---

## Étape 0 — Setup et exploration des données

On travaille avec le dataset **Cats vs Dogs** : des photos de chats et de chiens.
Notre objectif : construire un modèle qui distingue les deux.

### 0.1 — Installer et importer les librairies

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

### 0.2 — Télécharger le dataset

On utilise un sous-ensemble du dataset Cats vs Dogs (environ 3000 images).
C'est suffisant pour notre expérimentation et ça reste rapide sur Colab free tier.

In [ ]:
# Télécharger le dataset
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /content/catsdogs.zip

import zipfile
with zipfile.ZipFile('/content/catsdogs.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

base_dir = '/content/cats_and_dogs_filtered/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

print("Dataset téléchargé !")
print(f"Images d'entraînement (chats) : {len(os.listdir(os.path.join(train_dir, 'cats')))}")
print(f"Images d'entraînement (chiens) : {len(os.listdir(os.path.join(train_dir, 'dogs')))}")
print(f"Images de validation (chats) : {len(os.listdir(os.path.join(validation_dir, 'cats')))}")
print(f"Images de validation (chiens) : {len(os.listdir(os.path.join(validation_dir, 'dogs')))}")

### 0.3 — Charger les images

In [ ]:
IMG_SIZE = (160, 160)
BATCH_SIZE = 32

train_dataset = keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

val_dataset = keras.utils.image_dataset_from_directory(
    validation_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

class_names = train_dataset.class_names
print(f"Classes : {class_names}")

### 🤖 GEMINI — Visualiser les données

Demandez à Gemini :

> *"Génère du code pour afficher une grille de 12 images aléatoires du dataset `train_dataset` avec leurs labels ('cats' ou 'dogs') en titre. Utilise matplotlib avec une grille 3x4."*

Ensuite, demandez-lui :

> *"Explique-moi ce que fait chaque ligne de ce code."*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI



### ❓ QUESTION

En observant les images :
- Les images sont-elles toutes de la même taille ? de la même qualité ?
- Y a-t-il des images "difficiles" (animal de dos, mal cadré, plusieurs animaux) ?
- Pensez-vous qu'un humain ferait 100% de bonnes réponses sur ce dataset ?

*Écrivez vos observations ci-dessous :*

*(vos observations ici)*

### 0.4 — Optimiser le chargement des données

Cette cellule optimise le chargement des images pour accélérer l'entraînement.
`prefetch` prépare le batch suivant pendant que le GPU traite le batch courant.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)

---

## Étape 1 — Baseline : un petit CNN entraîné from scratch

Avant d'utiliser le Transfer Learning, on va d'abord essayer de construire un modèle **à partir de zéro** (from scratch). C'est notre **baseline** — le point de référence.

On construit un petit CNN (Convolutional Neural Network) avec 3 couches de convolution.

### 1.1 — Normaliser les pixels

Les pixels d'une image ont des valeurs entre 0 et 255. On les ramène entre 0 et 1 pour faciliter l'entraînement.

In [ ]:
normalization_layer = layers.Rescaling(1./255)

### 1.2 — Construire un petit CNN from scratch

In [ ]:
scratch_model = keras.Sequential([
    # Normalisation des pixels (0-255 → 0-1)
    normalization_layer,

    # Bloc 1 : détecter des features simples (bords, textures)
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D(2, 2),

    # Bloc 2 : détecter des features plus complexes (formes)
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    # Bloc 3 : features encore plus abstraites
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    # Classification
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # 1 sortie : chat (0) ou chien (1)
])

scratch_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

scratch_model.summary()

### 🤖 GEMINI — Comprendre l'architecture

Demandez à Gemini :

> *"Explique-moi le `model.summary()` ci-dessus. Que signifient les colonnes Output Shape et Param # ? Combien de paramètres ce modèle a-t-il au total ?"*

### 1.3 — Entraîner le modèle from scratch

On entraîne pendant 10 époques. Une époque = le modèle a vu toutes les images une fois.

In [ ]:
print("Entraînement du modèle from scratch...")
print("(cela prend environ 2-3 minutes)")

history_scratch = scratch_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

### 1.4 — Visualiser les résultats

In [ ]:
def plot_training(history, title):
    """Affiche les courbes d'accuracy et de loss pour train et validation."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Accuracy
    ax1.plot(history.history['accuracy'], label='Train', linewidth=2)
    ax1.plot(history.history['val_accuracy'], label='Validation', linewidth=2)
    ax1.set_title(f'{title} — Accuracy')
    ax1.set_xlabel('Époque')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([0.4, 1.0])

    # Loss
    ax2.plot(history.history['loss'], label='Train', linewidth=2)
    ax2.plot(history.history['val_loss'], label='Validation', linewidth=2)
    ax2.set_title(f'{title} — Loss')
    ax2.set_xlabel('Époque')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

plot_training(history_scratch, "CNN from scratch")

### ❓ QUESTION

Observez les courbes :
- Quelle est l'accuracy sur les données de validation à la dernière époque ?
- La courbe de train et la courbe de validation divergent-elles ? Si oui, que signifie cette divergence ?
- Ce modèle est-il satisfaisant pour distinguer un chat d'un chien ?

*Rappel : si le train monte mais la validation stagne ou descend, c'est de l'**overfitting** — le modèle mémorise les images d'entraînement au lieu d'apprendre à généraliser.*

*(vos observations ici)*

---

## Étape 2 — Transfer Learning avec MobileNetV2

Le modèle from scratch n'est pas terrible. Normal : on a seulement ~2000 images d'entraînement et un petit réseau.

Maintenant, on va utiliser le **Transfer Learning** : réutiliser un modèle déjà entraîné sur **1,4 million d'images** (ImageNet) et l'adapter à notre problème.

Le modèle choisi est **MobileNetV2** :
- Léger (~3,4 millions de paramètres)
- Rapide à exécuter (conçu pour les appareils mobiles)
- Très performant malgré sa taille

### 2.1 — Charger le modèle pré-entraîné

In [ ]:
# Charger MobileNetV2 pré-entraîné sur ImageNet
# include_top=False : on retire la dernière couche (qui classifie 1000 catégories ImageNet)
# On n'en veut pas : nous on a seulement 2 catégories (chat / chien)

base_model = keras.applications.MobileNetV2(
    input_shape=(160, 160, 3),
    include_top=False,          # on retire la tête de classification
    weights='imagenet'          # on charge les poids pré-entraînés
)

print(f"Nombre de couches dans MobileNetV2 : {len(base_model.layers)}")
print(f"Nombre de paramètres : {base_model.count_params():,}")

### 🤖 GEMINI — Comprendre MobileNetV2

Demandez à Gemini :

> *"Qu'est-ce que MobileNetV2 ? Sur quel dataset a-t-il été entraîné ? Combien de classes peut-il reconnaître à l'origine ? Pourquoi utilise-t-on `include_top=False` ?"*

### 2.2 — Geler toutes les couches

C'est l'étape clé du Transfer Learning. On **gèle** toutes les couches du modèle pré-entraîné pour que leurs poids ne soient pas modifiés pendant notre entraînement.

Ces poids ont été appris sur 1,4 million d'images. Ils savent déjà reconnaître des bords, des textures, des formes, des objets. On ne veut pas perdre tout ça.

In [ ]:
# GELER toutes les couches du modèle pré-entraîné
base_model.trainable = False

# Vérification
print(f"Paramètres totaux : {base_model.count_params():,}")
print(f"Paramètres entraînables : {sum(tf.keras.backend.count_params(w) for w in base_model.trainable_weights):,}")
print(f"Paramètres gelés : {sum(tf.keras.backend.count_params(w) for w in base_model.non_trainable_weights):,}")

### ❓ QUESTION

Combien de paramètres sont entraînables après le gel ? Pourquoi ?

*(votre réponse ici)*

### 2.3 — Ajouter notre tête de classification

On a retiré la tête de MobileNetV2 (qui classifiait 1000 catégories). On la remplace par la nôtre : une simple couche qui classifie **chat ou chien**.

In [ ]:
# Pré-traitement spécifique à MobileNetV2
# Ce modèle attend des pixels entre -1 et 1 (pas entre 0 et 255)
preprocess_input = keras.applications.mobilenet_v2.preprocess_input

# Construire le modèle complet
inputs = keras.Input(shape=(160, 160, 3))

# 1. Pré-traitement
x = preprocess_input(inputs)

# 2. Le modèle pré-entraîné (gelé) extrait les features
x = base_model(x, training=False)

# 3. Convertir les features en un vecteur
x = layers.GlobalAveragePooling2D()(x)

# 4. Un peu de dropout pour éviter l'overfitting
x = layers.Dropout(0.2)(x)

# 5. La couche finale : 1 neurone = chat (0) ou chien (1)
outputs = layers.Dense(1, activation='sigmoid')(x)

# Assembler le tout
tl_model = keras.Model(inputs, outputs)

tl_model.summary()

### 🤖 GEMINI — Comprendre l'architecture

Demandez à Gemini :

> *"Dans le code ci-dessus, explique-moi le rôle de `GlobalAveragePooling2D` et de `Dropout(0.2)`. Pourquoi les utilise-t-on ici ?"*

> *"Combien de paramètres sont entraînables dans ce modèle ? Compare avec le modèle from scratch."*

### 2.4 — Compiler et entraîner

In [ ]:
tl_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Entraînement du modèle Transfer Learning...")
print("(cela prend environ 1-2 minutes)")

history_tl = tl_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

### 2.5 — Visualiser les résultats

In [ ]:
plot_training(history_tl, "Transfer Learning — MobileNetV2")

---

## Étape 3 — Comparer les deux approches

C'est le moment clé. On met côte à côte les résultats du modèle from scratch et du Transfer Learning.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history_scratch.history['val_accuracy'], label='From scratch', linewidth=2, linestyle='--')
axes[0].plot(history_tl.history['val_accuracy'], label='Transfer Learning', linewidth=2)
axes[0].set_title('Accuracy sur la validation')
axes[0].set_xlabel('Époque')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0.4, 1.0])

# Loss
axes[1].plot(history_scratch.history['val_loss'], label='From scratch', linewidth=2, linestyle='--')
axes[1].plot(history_tl.history['val_loss'], label='Transfer Learning', linewidth=2)
axes[1].set_title('Loss sur la validation')
axes[1].set_xlabel('Époque')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Résumé chiffré
print("\n" + "="*50)
print("RÉSUMÉ")
print("="*50)
print(f"From scratch   — Validation accuracy : {history_scratch.history['val_accuracy'][-1]:.1%}")
print(f"Transfer Learning — Validation accuracy : {history_tl.history['val_accuracy'][-1]:.1%}")
print(f"")
print(f"From scratch   — Paramètres entraînés : {scratch_model.count_params():,}")
print(f"Transfer Learning — Paramètres entraînés : {tl_model.count_params() - base_model.count_params():,} (sur {tl_model.count_params():,} total)")

### ❓ QUESTIONS

Observez attentivement les courbes et les chiffres :

1. **Performance** : quelle est la différence d'accuracy entre les deux modèles ?
2. **Démarrage** : à quelle accuracy le modèle Transfer Learning commence-t-il dès la première époque ? Pourquoi démarre-t-il si haut ?
3. **Overfitting** : lequel des deux modèles montre le plus de signes d'overfitting ?
4. **Paramètres** : combien de paramètres entraîne-t-on dans chaque cas ? Qu'est-ce que cela implique en termes de temps de calcul ?
5. **Conclusion** : pourquoi le Transfer Learning fonctionne-t-il mieux avec si peu de données ?

*(vos réponses ici)*

---

## Étape 4 — Expérimenter

Maintenant c'est à vous de jouer. Choisissez **une ou plusieurs** expérimentations parmi les suivantes.

Pour chaque expérimentation :
1. Demandez à Gemini de générer le code
2. Lisez le code et demandez-lui de vous l'expliquer
3. Exécutez et observez les résultats
4. Notez vos conclusions

---

### Expérimentation A — Essayer un autre modèle pré-entraîné

MobileNetV2 n'est pas le seul modèle disponible. Essayez avec **EfficientNetB0** ou **ResNet50**.

### 🤖 GEMINI

Demandez à Gemini :

> *"Réécris le code de l'étape 2 en remplaçant MobileNetV2 par EfficientNetB0 (ou ResNet50). Garde la même structure : charger le modèle pré-entraîné sans la couche de sortie, geler les couches, ajouter GlobalAveragePooling2D, Dropout et une couche Dense sigmoid. Adapte la fonction de pré-traitement au modèle choisi."*

Puis demandez :

> *"Quelle est la différence entre MobileNetV2 et EfficientNetB0 ? Lequel a le plus de paramètres ?"*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI



### ❓ QUESTION

Comparez les résultats avec MobileNetV2 :
- L'accuracy est-elle meilleure, moins bonne, ou similaire ?
- L'entraînement a-t-il été plus long ?
- Le modèle est-il plus gros (plus de paramètres) ?

*(vos observations ici)*

---

### Expérimentation B — Ajouter du Data Augmentation

Le Data Augmentation crée des variantes des images d'entraînement (rotation, retournement, zoom) pour "augmenter" artificiellement la taille du dataset et réduire l'overfitting.

### 🤖 GEMINI

Demandez à Gemini :

> *"Ajoute des couches de Data Augmentation au modèle de Transfer Learning MobileNetV2. Utilise RandomFlip horizontal, RandomRotation de 0.2, et RandomZoom de 0.2. Place ces couches avant le preprocess_input. Montre-moi aussi le code pour afficher 9 versions augmentées d'une même image."*

Puis demandez :

> *"Pourquoi le Data Augmentation aide-t-il à réduire l'overfitting ?"*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI
# 1. Afficher les images augmentées



In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI
# 2. Modèle avec Data Augmentation + entraînement



### ❓ QUESTION

- L'écart entre train et validation a-t-il diminué par rapport au modèle sans augmentation ?
- L'accuracy de validation a-t-elle changé ?

*(vos observations ici)*

---

### Expérimentation C — Changer le nombre d'époques

On a entraîné pendant 10 époques. Que se passe-t-il si on entraîne plus longtemps ? Ou moins longtemps ?

### 🤖 GEMINI

Demandez à Gemini :

> *"Reprends le modèle Transfer Learning MobileNetV2 de l'étape 2 et entraîne-le pendant 3 époques, puis dans un second temps pendant 20 époques. Affiche les courbes de chaque entraînement."*

Puis demandez :

> *"À quoi sert l'EarlyStopping dans Keras ? Comment l'utiliser pour arrêter l'entraînement automatiquement quand le modèle n'apprend plus ?"*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI



### ❓ QUESTION

- À partir de quelle époque le modèle n'apprend plus (la validation accuracy stagne) ?
- À partir de quelle époque commence l'overfitting ?
- Quel serait le nombre idéal d'époques pour ce modèle ?

*(vos observations ici)*

---

## Étape 5 — Tester le modèle sur vos propres images

C'est le moment de vérité ! On va utiliser notre modèle pour classifier des images de chats et de chiens que vous choisissez.

### 5.1 — Uploader une image

Exécutez la cellule ci-dessous, puis uploadez une image de chat ou de chien depuis votre ordinateur.

In [ ]:
from google.colab import files

print("Uploadez une image de chat ou de chien :")
uploaded = files.upload()

### 5.2 — Prédiction

Le modèle va analyser votre image et dire s'il pense que c'est un chat ou un chien.

In [ ]:
for filename in uploaded.keys():
    # Charger et redimensionner l'image
    img = keras.utils.load_img(filename, target_size=(160, 160))
    img_array = keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Ajouter la dimension batch

    # Prédiction
    prediction = tl_model.predict(img_array)
    score = prediction[0][0]

    # Afficher le résultat
    plt.figure(figsize=(6, 6))
    plt.imshow(keras.utils.load_img(filename))
    plt.axis('off')

    if score > 0.5:
        label = "🐕 CHIEN"
        confidence = score
    else:
        label = "🐈 CHAT"
        confidence = 1 - score

    plt.title(f"{label}\nConfiance : {confidence:.1%}", fontsize=16)
    plt.show()

### 🤖 GEMINI — Aller plus loin

Demandez à Gemini :

> *"Modifie le code de prédiction pour qu'il puisse tester plusieurs images d'un coup et afficher les résultats dans une grille 2x2 avec les prédictions et la confiance."*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI



### ❓ QUESTIONS

- Le modèle a-t-il correctement classifié vos images ?
- Essayez avec une image ambiguë (un chat qui ressemble à un chien, ou un animal vu de loin). Que se passe-t-il ?
- Essayez avec une image qui n'est ni un chat ni un chien (un oiseau, une voiture, un paysage). Que prédit le modèle ? Pourquoi ?

*(vos observations ici)*

---

## Étape 6 — Synthèse

Vous avez construit et comparé deux approches pour classifier des images :

| | From scratch | Transfer Learning |
|---|---|---|
| **Principe** | Tout entraîner à partir de zéro | Réutiliser un modèle pré-entraîné |
| **Données nécessaires** | Beaucoup | Peu |
| **Temps d'entraînement** | Plus long | Plus court |
| **Performance** | Limitée | Élevée |
| **Paramètres entraînés** | Tous | Seulement la tête |

### ❓ QUESTIONS FINALES

1. Dans votre domaine professionnel, imaginez un cas d'usage où le Transfer Learning serait utile. Décrivez-le en 2-3 phrases.

2. Un collègue vous dit : *"J'ai 500 images de pièces industrielles défectueuses et je veux entraîner un modèle de détection from scratch."* Que lui conseillez-vous ?

3. Qu'est-ce qui vous a le plus surpris dans cet atelier ?

*(vos réponses ici)*

---

## Pour aller plus loin (optionnel)

Si vous avez du temps et de la curiosité, voici quelques pistes d'exploration. Utilisez Gemini pour vous guider.

- **Autre dataset** : demandez à Gemini de charger le dataset [Flowers](https://www.tensorflow.org/datasets/catalog/tf_flowers) (5 catégories de fleurs) et adaptez le modèle Transfer Learning
- **Visualiser les features** : demandez à Gemini de vous montrer ce que "voit" chaque couche du modèle (feature maps)
- **Matrice de confusion** : demandez à Gemini de générer une matrice de confusion pour voir en détail les erreurs du modèle
- **Comparer 3 modèles** : MobileNetV2, EfficientNetB0, ResNet50 — sur le même dataset, mêmes conditions, dans un tableau récapitulatif

In [ ]:
# VOTRE EXPLORATION ICI

